<a href="https://colab.research.google.com/github/siyeon-Hwang/aiffel_exp/blob/main/%5BNLP-05%5D%20Transformer%20Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 목차
***
1. 데이터 다운로드
2. 데이터 정제 및 토큰화
  1. 중복 제거
  2. 정제하기
  3. 토큰화
  4. 길이 제한 및 텐서 변환
3. 모델 설계
4. 훈련하기
  1. transformer 선언
  2. learning_rate 및 optimizer 선언
  3. Loss 함수 정의
  4. train_step 함수 정의
  5. 학습 진행
5. 회고

## 라이브러리 버전 확인
사용할 라이브러리 버전을 둘러보고,  
필요 라이브러리를 가져온다.

In [ ]:
import tensorflow
import numpy
import matplotlib

print(tensorflow.__version__)
print(numpy.__version__)
print(matplotlib.__version__)

2.8.0
1.21.5
3.2.2


In [ ]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

import time
import re
import io
import os
import random

import seaborn # Attention 시각화

### 나눔 글꼴 설치하기
[구글 코랩 한글 깨짐 현상 해결 방법](https://teddylee777.github.io/colab/colab-korean)

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv 
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

%config InlineBackend.figure_format = 'retina'

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager.findfont(font)

print("슝=3")

슝=3


### SentencePiece 다운로드

In [ ]:
# SentencePiece 다운로드
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 4.2 MB/s 


# Step 1. 데이터 다운로드
아래 링크에서 korean-english-park.train.tar.gz를 사용할 예정이다.
- [jungyeul/korean-parallel-corpora](https://github.com/jungyeul/korean-parallel-corpora/tree/master/korean-english-news-v1)

In [ ]:
#1. 경로 바꾸기
%cd '/content/drive/MyDrive/GoingDeeper_NLP/data/10.transformer_translator'

/content/drive/MyDrive/GoingDeeper_NLP/data/10.transformer_translator


In [ ]:
#2. 데이터 다운로드
!wget https://raw.githubusercontent.com/jungyeul/korean-parallel-corpora/master/korean-english-news-v1/korean-english-park.train.tar.gz

--2022-04-11 16:48:58--  https://raw.githubusercontent.com/jungyeul/korean-parallel-corpora/master/korean-english-news-v1/korean-english-park.train.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8718893 (8.3M) [application/octet-stream]
Saving to: ‘korean-english-park.train.tar.gz’

korean-english-park 100%[===================>]   8.31M  48.2MB/s    in 0.2s    

2022-04-11 16:48:59 (48.2 MB/s) - ‘korean-english-park.train.tar.gz’ saved [8718893/8718893]



In [ ]:
#3. tar.gz 파일 압축 해제
!tar -xzvf korean-english-park.train.tar.gz

korean-english-park.train.en
korean-english-park.train.ko


# Step 2. 데이터 정제 및 토큰화


### (1) 데이터 중복 제거
set 데이터 형을 활용해 중복된 데이터를 제거한다. 

데이터의 병렬 쌍이 흐트러지지 않게 주의하고,  
 중복을 제거한 데이터를 cleaned_corpus에 저장한다.

In [ ]:
data_dir = '/content/drive/MyDrive/GoingDeeper_NLP/data/10.transformer_translator'
kor_path = data_dir + '/korean-english-park.train.ko'
eng_path = data_dir + '/korean-english-park.train.en'

# 데이터 정제 및 토큰화
def clean_corpus(kor_path, eng_path):
  with open(kor_path, 'r') as f: kor = f.read().splitlines()
  with open(eng_path, 'r') as f: eng = f.read().splitlines()
  
  assert len(kor) == len(eng)

  cleaned_corpus = list(set(zip(kor, eng)))

  return cleaned_corpus

cleaned_corpus = clean_corpus(kor_path, eng_path)

In [ ]:
cleaned_corpus[:3]

[('그러나 의회 지도자들에게 보내는 서안을 통해 부시 미 대통령은 협상은 반드시 마무리될 것이라고 확신했다.',
  'But in a letter to Congressional leaders, President Bush confirmed that an agreement had been reached.'),
 ('발코니에서 바라본 라펠로는 너무나 넓어 미국의 작가인 고르 비달은 “세계 에서 풍경이 가장 아름다운 곳”이라고 말했다.',
  'The view from the balcony is so wide that the American writer Gore Vidal who owned a nearby villa once defined it as "the most beautiful in the world."'),
 ('교전에는 연합군 항공기가 동원됐으며 연합군은 적의 위치를 정확히 파악할 수 있는 정밀한 유도장치가 부착된 무기를 사용, 맹공격에 나섰다.',
  'Coalition aircraft were called in and pounded "positively identified enemy positions using precision guided munitions.')]

### (2) 정제 함수 정의
정제 함수를 아래 조건을 만족하게 정의한다.
- 모든 입력을 소문자로 변환
- 알파벳, 문장부호, 한글만 남기고 모두 제거
- 문장부호 양옆에 공백 추가
- 문장 앞뒤의 불필요한 공백 제거

In [ ]:
def preprocess_sentence(sentence):

  sentence = sentence.lower().strip()
  sentence = re.sub(r'([?.!,])', r' \1 ', sentence)
  sentence = re.sub(r'([" "]+)', " ", sentence)
  
  reg = re.compile('[^a-zㄱ-ㅎㅏ-ㅣ가-힣,?.! ]')
  sentence = reg.sub('', sentence)

  sentence = sentence.strip()

  return sentence
  

### (3) 토큰화 
한글 말뭉치(kor_corpus)와 영문 말뭉치(eng_corpus)를 각각 **분리** 후, **정제** 하여  **토큰화** 를 진행한다.  

첨부한 공식 사이트를 참고하여 아래 조건을 만족하는 generate_tokenizer() 함수를 정의한다.  
최종적으로 ko_tokenizer와 en_tokenizer를 얻는다.  
en_tokenizer에는 set_encode_extra_options('bos:eos') 함수를 실행해 타겟 입력이 문장의 시작토큰과 끝토큰을 포함할 수 있게 한다. 
- [google/sentencepiece](https://github.com/google/sentencepiece)
  - 단어 사전을 매개변수로 받아 원하는 크기의 사전을 정의할 수 있게 한다.(기본: 20000)
  - 학습 후 저장된 model 파일을 SentencePieceProcessor() 클래스에 Load() 한 후 반환한다.
  - 특수 토큰의 인덱스를 아래와 동일하게 지정한다.  
  <PAD\>:0 / <BOS\>:1 / <EOS\>:2 / <UNK\>:3

In [ ]:
import sentencepiece as spm

In [ ]:
def generate_tokenizer(corpus,
                       vocab_size,
                       model_type='unigram',
                       lang="ko",
                       pad_id = 0,
                       bos_id = 1,
                       eos_id = 2,
                       unk_id = 3):
  input_file = f'{lang}_spm_input.txt'

  with open(input_file, 'w', encoding='utf-8') as f:
    for row in corpus:
      f.write(str(row) + '\n')

  sp_model_root = 'sentencepiece'
  if not os.path.isdir(sp_model_root): os.mkdir(sp_model_root)

  prefix = '%s_tokenizer_%s' % (lang, model_type+str(vocab_size))
  prefix = os.path.join(sp_model_root, prefix) # 저장될 tokenizer 모델 이름
  
  spm.SentencePieceTrainer.Train('--pad_id={} --bos_id={} --eos_id={} --unk_id={} \
  --input={} --model_prefix={} --vocab_size={} --model_type={}'.format(pad_id, bos_id,\
                                                                       eos_id, unk_id, input_file,\
                                                                       prefix, vocab_size,
                                                                       model_type))

  tokenizer = spm.SentencePieceProcessor()
  tokenizer.Load(f'{prefix}.model')

  return tokenizer


In [ ]:
SRC_VOCAB_SIZE = TGT_VOCAB_SIZE = 20000

eng_corpus = []
kor_corpus = []

for pair in cleaned_corpus:
  k, e = pair[0], pair[1]

  kor_corpus.append(preprocess_sentence(k))
  eng_corpus.append(preprocess_sentence(e))

ko_tokenizer = generate_tokenizer(kor_corpus, SRC_VOCAB_SIZE, 'unigram', "ko")
en_tokenizer = generate_tokenizer(eng_corpus, TGT_VOCAB_SIZE, 'unigram', "en")
en_tokenizer.set_encode_extra_options('bos:eos')

True

In [ ]:
kor_corpus[:3]

['그러나 의회 지도자들에게 보내는 서안을 통해 부시 미 대통령은 협상은 반드시 마무리될 것이라고 확신했다 .',
 '발코니에서 바라본 라펠로는 너무나 넓어 미국의 작가인 고르 비달은 세계 에서 풍경이 가장 아름다운 곳이라고 말했다 .',
 '교전에는 연합군 항공기가 동원됐으며 연합군은 적의 위치를 정확히 파악할 수 있는 정밀한 유도장치가 부착된 무기를 사용 , 맹공격에 나섰다 .']

In [ ]:
ko_tokenizer.EncodeAsIds(kor_corpus[0])

[69,
 475,
 1211,
 62,
 4273,
 5923,
 5,
 94,
 95,
 59,
 100,
 461,
 9,
 279,
 3435,
 2154,
 119,
 48,
 7273,
 4]

### (4) 길이 제한 및 텐서 변환
토크나이저를 활용해 **토큰의 길이가 50 이하** 인 데이터를 선별하여 src_corpus와 tgt_corpus를 각각 구축하고,  
텐서 enc_train과 dec_train으로 변환한다.  
(❗모든 데이터를 사용할 경우 학습에 굉장히 오랜 시간이 걸린다.)

In [ ]:
from tqdm.notebook import tqdm  # process 과정을 보기 위함

src_corpus = []
tgt_corpus = []

assert len(kor_corpus) == len(eng_corpus)

# 토큰의 길이가 50 이하인 문장만 남긴다.
for idx in tqdm(range(len(kor_corpus))):
  src = ko_tokenizer.EncodeAsIds(kor_corpus[idx])
  tgt = en_tokenizer.EncodeAsIds(eng_corpus[idx])

  if (len(src) <= 50) and (len(tgt) <= 50):
    src_corpus.append(src)
    tgt_corpus.append(tgt)

# 패딩 처리를 완료하여 학습용 데이터를 완성한다.
enc_train = tf.keras.preprocessing.sequence.pad_sequences(src_corpus,
                                                          padding='post')
dec_train = tf.keras.preprocessing.sequence.pad_sequences(tgt_corpus,
                                                          padding='post')

  0%|          | 0/78968 [00:00<?, ?it/s]

# Step 3. 모델 설계
1. 내부 모듈 구현  
  (1) Positional Encoding  
  (2) Multi-Head Attention  
  (3) Position-wise Feed-Forward Network  

2. 모듈 조립  
  (1) Encoder 레이어  
  (2) Decoder 레이어  
  (3) Transformer  

## 1) 내부 모듈 구현

### (1) Positional Encoding

In [ ]:
def positional_encoding(pos, d_model):
  def cal_angle(position, i):
    return position / np.power(10000, int(i) / d_model)

  def get_posi_angle_vec(position):
    return [cal_angle(position, i) for i in range(d_model)]

  sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])
  sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
  sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])
  return sinusoid_table

print('슝=3')

슝=3


### (2) Multi-Head Attention

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    self.depth = d_model // self.num_heads 

    self.W_q = tf.keras.layers.Dense(d_model) #Linear Layer
    self.W_k = tf.keras.layers.Dense(d_model)
    self.W_v = tf.keras.layers.Dense(d_model)

    self.linear = tf.keras.layers.Dense(d_model)

  
  def scaled_dot_product_attention(self, Q, K, V, mask):
    d_k = tf.cast(K.shape[-1], tf.float32) 

    QK = tf.matmul(Q, K, transpose_b =True)   # QK 내적

    scaled_qk = QK / tf.math.sqrt(d_k)   # scaled 된 QK 값 구하기

    if mask is not None: scaled_qk += (mask * -1e9)  # 후에 나올 masking 처리

    attentions = tf.nn.softmax(scaled_qk, axis=-1)  # attention weights 값 구하기
    
    out = tf.matmul(attentions, V)   # attention 값을 V에 곱하기

    return out, attentions


  def split_heads(self, x):
    """
    Embedding을 Head의 수로 분할하는 함수

    x: [ batch x length x emb ]
    return: [ batch x length x heads x self.depth ]
    """
    batch_size = x.shape[0]
    split_x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])  # 4차원 transpose를 왜 하지??

    return split_x

  
  def combine_heads(self, x):
    """
    분할된 Embedding을 하나로 결합하는 함수

    x: [ batch x length x heads x self.depth ]
    return: [ batch x length x emb ]
    """
    batch_size = x.shape[0]
    combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
    combined_x = tf.reshape(combined_x, (batch_size, -1, self.d_model))

    return combined_x


  def call(self, Q, K, V, mask):
    # 1.  Linear_in(Q, K, V) -> WQ, WK, WV
    WQ = self.W_q(Q)
    WK = self.W_k(K)
    WV = self.W_v(V)

    # 2. Split Heads(WQ, WK, WV) -> WQ_split, WK_split, WV_split
    WQ_splits = self.split_heads(WQ)
    WK_splits = self.split_heads(WK)
    WV_splits = self.split_heads(WV)

    # 3. Scaled Dot Product Attention(WQ_split, WK_split, WV_split) -> out, attention_weights
    out, attention_weights = self.scaled_dot_product_attention(WQ_splits,
                                                               WK_splits,
                                                               WV_splits,
                                                               mask)
    
    # 4. Combine Heads(out) -> out
    out = self.combine_heads(out)

    # 5. Linear_out(out) -> out
    out = self.linear(out)

    return out, attention_weights

print('슝=3')

슝=3


### (3) Position-wise Feed-Forward Network

In [ ]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
  def __init__(self, d_model, d_ff):
    super(PoswiseFeedForwardNet, self).__init__()
    self.w_1 = tf.keras.layers.Dense(d_ff, activation='relu')
    self.w_2 = tf.keras.layers.Dense(d_model)

  def call(self, x):
    out = self.w_1(x)
    out = self.w_2(out)

    return out

print('슝=3')

슝=3


## 2) 모듈 조립하기

### (1) Encoder 레이어 구현

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, n_heads, d_ff, dropout):
    super(EncoderLayer, self).__init__()

    self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
    self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

    self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout = tf.keras.layers.Dropout(dropout)


  def call(self, x, mask):
    # Multi-Head Attention
    residual = x
    out = self.norm_1(x)
    out, enc_attn = self.enc_self_attn(out, out, out, mask)
    out = self.dropout(out)
    out += residual

    # Position-Wise Feed Forward Network
    residual = out
    out = self.norm_2(out)
    out = self.ffn(out)
    out = self.dropout(out)
    out += residual

    return out, enc_attn

print('슝=3')

슝=3


### (2) Decoder 레이어 구현

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, d_ff, dropout):
    super(DecoderLayer, self).__init__()

    self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
    self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

    self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

    self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout = tf.keras.layers.Dropout(dropout)


  def call(self, x, enc_out, causality_mask, padding_mask):
    # Masked Multi-Head Attention
    residual = x
    out = self.norm_1(x)
    out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
    out = self.dropout(out)
    out += residual

    # Multi-Head Attention
    residual = out
    out = self.norm_2(out)
    out, dec_enc_attn = self.enc_dec_attn(out, enc_out, enc_out, causality_mask)
    out = self.dropout(out)
    out += residual

    # Position-Wise Feed Forward Network
    residual = out
    out = self.norm_3(out)
    out = self.ffn(out)
    out = self.dropout(out)
    out += residual

    return out, dec_attn, dec_enc_attn

print('슝=3')

슝=3


In [ ]:
# Encoder 클래스 정의
class Encoder(tf.keras.Model):
  def __init__(self,
               n_layers,
               d_model,
               n_heads,
               d_ff,
               dropout):
    super(Encoder, self).__init__()
    self.n_layers = n_layers
    self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)]


  def call(self, x, mask):
    out = x

    enc_attns = list()
    for i in range(self.n_layers):
      out, enc_attn = self.enc_layers[i](out, mask)
      enc_attns.append(enc_attn)

    return out, enc_attns

print('슝=3')

슝=3


In [ ]:
# Decoder 클래스 정의
class Decoder(tf.keras.Model):
  def __init__(self,
               n_layers,
               d_model,
               n_heads,
               d_ff,
               dropout):
    super(Decoder, self).__init__()
    self.n_layers = n_layers
    self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)]


  def call(self, x, enc_out, causality_mask, padding_mask):
    out = x
    
    dec_attns = list()
    dec_enc_attns = list()
    for i in range(self.n_layers):
      out, dec_attn, dec_enc_attn = self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

      dec_attns.append(dec_attn)
      dec_enc_attns.append(dec_enc_attn)

    return out, dec_attns, dec_enc_attns

  print('슝=3')

슝=3


### (3) Transformer 

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self,
               n_layers,
               d_model,
               n_heads,
               d_ff,
               src_vocab_size,
               tgt_vocab_size,
               pos_len,
               dropout=0.2,
               shared=True):
    super(Transformer, self).__init__()
    self.d_model = tf.cast(d_model, tf.float32)
    
    # 1. Embedding Layer 정의
    self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
    self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

    # 2. Positional Encoding 정의
    self.pos_encoding = positional_encoding(pos_len, d_model)

    # 3. Dropout 정의
    self.dropout = tf.keras.layers.Dropout(dropout) 

    # 4. Encoder / Decoder 정의
    self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
    self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)
    
    # 5. Output Linear 정의
    self.fc = tf.keras.layers.Dense(tgt_vocab_size)

    # 6. Shared Weights
    self.shared = shared

    if shared: self.fc.set_weights(tf.transpose(self.dec_emb.weights))


  def embedding(self, emb, x):
    """
    입력된 정수 배열을 Embedding + Pos Encoding
    + Shared 일 경우 Scaling 작업 포함

    x: [batch x length]
    return: [batch x length x emb]
    """
    seq_len = x.shape[1]
    out = emb(x)

    if self.shared: out *= tf.math.sqrt(self.d_model)

    out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
    out = self.dropout(out)

    return out


  def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
    """
    아래 순서에 따라 작성

    Step 1: Embedding(enc_in, dec_in) -> enc_in, dec_in
    Step 2: Encoder(enc_in, enc_mask) -> enc_out, enc_attns
    Step 3: Decoder(dec_in, enc_out, mask) -> dec_out, dec_attns, dec_enc_attns
    Step 4: Out Linear(dec_out) -> logits
    """
    enc_in = self.embedding(self.enc_emb, enc_in)
    dec_in = self.embedding(self.dec_emb, dec_in)

    enc_out, enc_attns = self.encoder(enc_in, enc_mask)

    dec_out, dec_attns, dec_enc_attns = \
    self.decoder(dec_in, enc_out, causality_mask, dec_mask)

    logits = self.fc(dec_out)

    return logits, enc_attns, dec_attns, dec_enc_attns

print('슝=3')

슝=3


# Step 4. 훈련하기
1. 2 layer를 가지는 Transformer 선언.
2. Learning Rate Scheduler 선언 및 Adam Optimizer 선언
3. Loss 함수 정의
4. train_step 함수 정의 및 Mask 생성
5. 학습 진행

### (1) Transformer 선언
2 Layer를 가지는 Transformer 선언하기. 
(하이퍼 파라미터는 자유롭게 조절)

In [ ]:
transformer = Transformer(n_layers=2,
                          d_model=512,
                          n_heads=8,
                          d_ff=2048,
                          src_vocab_size=SRC_VOCAB_SIZE,
                          tgt_vocab_size=TGT_VOCAB_SIZE,
                          pos_len=50,
                          dropout=0.2,
                          shared=True)

### (2) Learning Rate Scheduler 및 Adam Optimizer 선언
논문에서 사용한 것과 동일한 Learning Rate Scheduler를 선언하고,   
이를 포함하는 Adam Optimizer 를 선언한다. (Optimizer의 파라미터 역시 논문과 동일하게 설정한다.)

In [ ]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps = 4000):
    super(LearningRateScheduler, self).__init__()
    self.d_model = d_model
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = step ** -0.5
    arg2 = step * (self.warmup_steps ** -1.5)

    return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)


learning_rate = LearningRateScheduler(512)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                     beta_1 = 0.9,
                                     beta_2 = 0.98,
                                     epsilon = 1e-9)

print('슝=3')

슝=3


### (3) Loss 함수 정의
Sequence-to-sequence 모델에서 사용했던 Loss와 유사하되, **Masking 되지 않은 입력의 개수로 Scaling** 하는 과정을 추가한다. (트랜스포머가 모든 입력에 대한 Loss를 한 번에 구하기 때문)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits = True, reduction='none'
)

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  #Masking 되지 않은 입력의 개수로 Scaling 하는 과정
  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

### (4) Masking 정의

In [ ]:
def generate_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32) 
  return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
  mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
  return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
  enc_mask = generate_padding_mask(src)
  dec_mask = generate_padding_mask(tgt)

  dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
  dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

  dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
  dec_mask = tf.maximum(dec_mask, dec_causality_mask)

  return enc_mask, dec_enc_mask, dec_mask

print('슝=3')

슝=3


### (5) train_step 함수 정의
**train_step 함수**를 정의하고, **입력 데이터에 알맞은 Mask를 생성**하고, 이를 모델에 전달하여 연산에서 사용할 수 있게 한다.

In [ ]:
@tf.function()
def train_step(src, tgt, model, optimizer):
  gold = tgt[:, 1:]

  enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt)

  # 계산된 loss에 tf.GradientTape()를 적용해 학습을 진행한다.
  with tf.GradientTape() as tape:
    predictions, enc_attns, dec_attns, dec_enc_attns = model(src, tgt, enc_mask, dec_enc_mask, dec_mask)

    loss = loss_function(gold, predictions[:, :-1])

    # 최종적으로 optimizer.apply_gradients()가 사용됩니다.
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

### (6) 학습 진행
매 Epoch 마다 제시된 예문에 대한 번역을 생성하고,   
번역이 잘 되면, 그 때의 하이퍼파라미터와 생성된 번역을 제출한다.

In [ ]:
# Attention 시각화 함수
def visualize_attention(src, tgt, enc_attns, dec_attns, dec_enc_attns):
  def draw(data, ax, x='auto', y = 'auto'):
    import seaborn
    seaborn.heatmap(data, 
                    square=True,
                    vmin=0.0, vmax=1.0,
                    cbar=False, ax=ax,
                    xticklabels=x,
                    yticklabels=y)
    
  for layer in range(0, 2, 1):
    fig, axs = plt.subplots(1, 4, figsize=(20,10))
    print('Encoder Layer', layer + 1)
    for h in range(4):
      draw(enc_attns[layer][0, h, :len(src), :len(src)], axs[h], src, src)
    plt.show()

  for layer in range(0, 2, 1):
    fig, axs = plt.subplots(1, 4, figsize=(20,10))
    print('Decoder Self Layer', layer+1)
    for h in range(4):
      draw(dec_attns[layer][0, h, :len(tgt), :len(tgt)], axs[h], tgt, tgt)
    plt.show()

    print('Decoder Src Layer', layer+1)
    fig, axs = plt.subplots(1, 4, figsize=(20,10))
    for h in range(4):
      draw(dec_enc_attns[layer][0, h, :len(tgt), :len(src)], axs[h], src, tgt)
    plt.show()

In [ ]:
# 번역 생성 함수
def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
  sentence = preprocess_sentence(sentence)

  pieces = src_tokenizer.encode_as_pieces(sentence)
  tokens = src_tokenizer.encode_as_ids(sentence)

  _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                         maxlen=enc_train.shape[-1],
                                                         padding = 'post')
  ids = []
  output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)
  for i in range(dec_train.shape[-1]):
    enc_padding_mask, combined_mask, dec_padding_mask = \
    generate_masks(_input, output)

    predictions, enc_attns, dec_attns, dec_enc_attns = \
    model(_input,
          output,
          enc_padding_mask,
          combined_mask,
          dec_padding_mask)
    
    predicted_id = \
    tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

    if tgt_tokenizer.eos_id() == predicted_id:
      result = tgt_tokenizer.decode_ids(ids)
      return pieces, result, enc_attns, dec_attns, dec_enc_attns

    ids.append(predicted_id)
    output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

  result = tgt_tokenizer.decode_ids(ids)

  return pieces, result, enc_attns, dec_attns, dec_enc_attns


In [ ]:
# 번역 생성 및 Attention 시각화 결합
def translate(sentence, model, src_tokenizer, tgt_tokenizer, plot_attention=False):
  pieces, result, enc_attns, dec_attns, dec_enc_attns = \
  evaluate(sentence, model, src_tokenizer, tgt_tokenizer)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  if plot_attention:
    visualize_attention(pieces, result.split(), enc_attns, dec_attns, dec_enc_attns)

In [ ]:
from tqdm import tqdm_notebook

BATCH_SIZE = 64
EPOCHS = 10

examples = [
            '오바마는 대통령이다.',
            '시민들은 도시 속에 산다.',
            '커피는 필요 없다.',
            '일곱 명의 사망자가 발생했다.'
]

for epoch in range(EPOCHS):
  total_loss = 0

  idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
  random.shuffle(idx_list)
  t = tqdm_notebook(idx_list)

  for (batch, idx) in enumerate(t):
    batch_loss, enc_attns, dec_attns, dec_enc_attns = \
    train_step(enc_train[idx:idx+BATCH_SIZE],
               dec_train[idx:idx+BATCH_SIZE],
               transformer,
               optimizer)
    
    total_loss += batch_loss

    t.set_description_str('Epoch %2d' % (epoch + 1))
    t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

  for example in examples:
    translate(example, transformer, ko_tokenizer, en_tokenizer)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1147 [00:00<?, ?it/s]

Input: 오바마는 대통령이다.
Predicted translation: obamas president .
Input: 시민들은 도시 속에 산다.
Predicted translation: they are the cities of the city .
Input: 커피는 필요 없다.
Predicted translation: caffein is not true .
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: the deadliest shootings were among the dead .


  0%|          | 0/1147 [00:00<?, ?it/s]

Input: 오바마는 대통령이다.
Predicted translation: president barack obama is in his office .
Input: 시민들은 도시 속에 산다.
Predicted translation: the city of the town of the city of the city of the city .
Input: 커피는 필요 없다.
Predicted translation: coffee gets a little bit of coffee .
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: the death toll from the deaths .


...

Input: 오바마는 대통령이다.
Predicted translation: obamas president , became a former president .
Input: 시민들은 도시 속에 산다.
Predicted translation: they want to go up today .
Input: 커피는 필요 없다.
Predicted translation: but the need to go .
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: seventeen people .


  0%|          | 0/1147 [00:00<?, ?it/s]

Input: 오바마는 대통령이다.
Predicted translation: if obama was a president .
Input: 시민들은 도시 속에 산다.
Predicted translation: they are the other cities to cities
Input: 커피는 필요 없다.
Predicted translation: coffee needs .
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: seven other people fatalities were killed .


  0%|          | 0/1147 [00:00<?, ?it/s]

Input: 오바마는 대통령이다.
Predicted translation: obama thens his then .
Input: 시민들은 도시 속에 산다.
Predicted translation: they are the other battle in the city .
Input: 커피는 필요 없다.
Predicted translation: but needs to need just because its need
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: seven people , with seven deaths sunday .


Translations
1. obama is a nominee .
2. the urbanization is in the city .
3. no coffee is needed .
4. seven people were killed 

Hyperparameters
- n_layers : 2
- d_model : 512
- n_heads : 8
- d_ff : 2048
- dropout : 0.2

Traning Parameters
- warmup Steps : 4000
- Batch Size : 64
- Epoch At : 5

SentencePiece model_type: **bpe**
***
확실히 sentencepiece 모델 타입 성능은 bpe가 unigram에 비해 훨씬 좋은 것 같다. 

위의 예측결과는 model_type이 bpe에서 unigram으로 바뀌었을 뿐인데 bpe의 translation 결과에 비해 말이 안 되는 번역이 많은 것을 볼 수 있다.   
앞으로 SentencePiece를 쓰게 되면 bpe로 고정시켜 사용할 것 같다. 

Seq2seq를 사용해서 번역기를 만들 때는 assert는 생각하지도 못했는데, 당연하다. 모르니까 쓸 수 없었다. 잘 알아두고 가야겠다.

모델설계까지는 어찌저찌 이해하고 짤 수 있을 것 같은데 훈련하기 부터, gradient tape와 learning rate scheduler 는 코드를 봐도 왜 이렇게 하는 것인가 이해가 필요할 것 같다. 

모델에서도 특정 코드가 어디에 쓰이고 있는지는 알겠지만 자잘한 라이브러리들, 수식들은 더 공부할 필요가 있어보인다. 
